# Descriptive summary: Multi-class experiment

In this notebook we build a cGAN model that generates multi-class observations for Benign, Bruteforce, Bot, DoS and DDoS classes. This includes:
                            

1.   Building a discriminator 
2.   Building a generator 
3.   Combining the generator and discriminator model to update the generator
4.   Generating datasets based on each generated saved generator model
5.   Comparing and saving the generated dataset with the most success 

Then we generate data with our best cGAN model and use it to train a Random Forest model. Thereafter, we compare the performance of the aforementioned model to that of Random Forest trained on a real dataset. A results notebook has been prepared with the results collated for our research. This can be found in : /RESEARCH-PROJECT/Multi_class_experiments/Results/scores.ipynb

Attack labels are numerically numbered as follows:

*   0 - Benign
*   1 - Brute force
*   2 - Bot
*   3 - DoS
*   4 - DDoS




In [ ]:
# connect to google drive for storing and test_ data and model 
from google.colab import drive

#  modules for importing and manipulating data
import pandas as pd 
import re 

# modules used throughout the gans process (mainly feeding noise and real/fake labels to generator)
import numpy as np 
from numpy.random import randint,randn
from numpy import(
    expand_dims,
    zeros,
    ones,
    asarray
)

# modules used to build gans (mainly diffrent types of layers, regularisation and optimisers)
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
      Input,
      BatchNormalization,
      Dense,
      Reshape,
      Flatten,
      LeakyReLU,
      Dropout,
      Lambda,
      Activation,
      Embedding,
      Concatenate,
      multiply
)



# modules for data transformation, sampling and modelling 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

# modules for scoring and evaluating performance
from sklearn.metrics import (
    roc_curve,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    accuracy_score,
    f1_score
)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import data from drive

Note: 


1. Data was previously downloaded from source: https://www.unb.ca/cic/datasets/ids-2017.html

2. Pre-processing: the data was pre-processed as was mentioned in the main text. 


3. Down sampled to only include 10k observations per attack class. This was due to the storage and computation constraints experienced when working with colab.

In [ ]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')

# import data from drive 
data_extension = '/content/drive/MyDrive/RESEARCH-PROJECT/Multi_class_experiments/Datasets/prepared201710k.PARQUET/prepared201710k.PARQUET'
df1 = pd.read_parquet(data_extension, engine='auto')

Mounted at /content/drive


# Prepare the data for Over-sampling 

In [ ]:
X = df1.drop(["attack_map"], axis=1) 
y = df1['attack_map']

# Over-sampling using Smote

under-represented data is over-sampled to 10k observations.

The use of SMOTE prevents the generation of duplicate observations and ensures a more uniform representation of attack classes.

In [ ]:
# use SMOTE to increase Bot and Bruteforce to 10k
sampler = {0:10000,1:10000,2:10000,3:10000,4:10000}
oversample = SMOTE(sampling_strategy=sampler)
X, y = oversample.fit_resample(X, y)

# Split data into training and testing samples

In [ ]:
# train, test split (75/25)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42,stratify=y
)

# Scale features to be between 0-1


Standardisation is a form of scaling that standardises all value ranges within our dataset to be between 0-1.

 In doing so, this:


1. speeds up convergence 
2. creates a more stable training process 


In [ ]:
# Scale all features to be between 0-1 this is to ensure all features are view with equal importance by the algorithm
scaler = StandardScaler()

# Make sure to only fit the scaler on the training data
X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_train.columns] = scaler.transform(X_test[X_train.columns])

# Build discriminator
1.   We have two input layers, one for the features and one for the labels.

2. An embedding layer is used to encode the label input into a 67-dimensional vector, i.e. one dimension per feature.

3.  We then use the multiply layer to condition features on the label embedding. This is referred to a model input.

4.  This "model input" is fed through 4 hidden layers.

5.   We also use dropout after the second and third layer, to discourage overfitting.

6.  Finally, we have one output layer that distinguishes between real and generated observations.

In [ ]:
# define the supervised discriminator model
def define_discriminator(out_shape=67, num_classes=5):
    # Initialiser that draws samples from a uniform distribution 
    init = 'he_uniform'

    # label input
    label = Input(shape=(1,), dtype='int64')

    # embed and flatten labels: we then get vector of shape 1 x output shape 
    label_embedding = Flatten()(Embedding(num_classes, out_shape)(label))

    # feed features through the second input layer 
    gen_sample = Input(shape=(out_shape,))

    # condition the discrimination of generated features
    model_input = multiply([gen_sample, label_embedding])

    # our first hidden layer and leaky relu activation
    fe = Dense(800, kernel_initializer=init)(model_input)
    fe = LeakyReLU(alpha=0.2)(fe)

    # our second hidden layer and leaky relu activation
    fe = Dense(512, kernel_initializer=init)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    # apply dropout
    fe = Dropout(0.4)(fe)

    # our third hidden layer and leaky relu activation
    fe = Dense(256, kernel_initializer=init)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    # apply dropout
    fe = Dropout(0.1)(fe)
    
     # our fourth hidden layer and leaky relu activation
    fe = Dense(128, kernel_initializer=init)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    # our output layer, with a single node (outputting the probability of an observation being real)
    fe = Dense(1)(fe)

    # sigmoid activation function
    out_layer= Activation('sigmoid')(fe)

    # model layer groups layers into an object with training and inference features.
    model = Model([gen_sample, label], out_layer)

    # compile model 
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5),metrics=[ tf.keras.metrics.Recall()])

    # save model architecture plot 
    dot_img_file = '/content/drive/MyDrive/RESEARCH-PROJECT/Multi_class_experiments/Models/MultiClass_discriminator.png'
    tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

    return model

# Build generator

1.   We have two input layers, one for the noise z and one for the labels.

2. An embedding layer is used to encode the label input into a 100-dimensional vector, i.e. one per latent dimension.

3.  we then use the multiply layer to condition the noise on "label_embedding". This is referred to as model input.

4.  This "model input" is fed through 4 hidden layers.

5.   We also use dropout after the second layer, to discourage overfitting.

6.  Finally, we have one output layer with 67 nodes, each outputting one feature.

7.  Note: we intentionally do not compile the generator model as it is not trained directly.


In [ ]:
def define_generator(latent_dim=100, out_shape=67,num_classes=5):
    # Initialiser that draws samples from a uniform distribution 
    init = 'he_uniform'

    # label input layer
    label = Input(shape=(1,), dtype='int64')

    # convert label into 100 dimensional vector
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

    # Noise z input layer
    noise = Input(shape=(latent_dim,))

    # We condition the generation of features
    model_input = multiply([noise, label_embedding])

    # our first hidden layer and leaky relu activation
    fe = Dense(128, kernel_initializer=init)(model_input)
    fe = LeakyReLU(alpha=0.2)(fe)

    # apply batch normalisation
    fe = BatchNormalization(momentum=0.8)(fe)

    # our second hidden layer and leaky relu activation
    fe = Dense(256, kernel_initializer=init)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    # apply batch normalisation
    fe = BatchNormalization(momentum=0.8)(fe)

    # our third hidden layer and leaky relu activation
    fe = Dense(512, kernel_initializer=init)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    # apply batch normalisation
    fe = BatchNormalization(momentum=0.8)(fe)

    # our fourth hidden layer and leaky relu activation
    fe = Dense(800, kernel_initializer=init)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
  
    # our output layer, with 67 nodes (one node per feature)
    out_layer= Dense(out_shape, activation='linear') (fe)

    # define the generator model
    model = Model([noise, label], out_layer) 
    return model

# Combine the generator and discriminator models to form cGAN

"define_cgan()" function combines our predefined generator and discriminator by taking them both as inputs.

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_cgan(g_model, d_model):

	# make weights in the discriminator not trainable
	d_model.trainable = False

	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input

	# get output from the generator model
	gen_output = g_model.output

	# connect output and label input from generator as inputs to discriminator
	cgan_output = d_model([gen_output, gen_label])
 
	# define cgan model as taking noise and labels and outputting a classification
	model = Model([gen_noise, gen_label], cgan_output)
 
	# compile model
	opt = Adam(lr=0.001, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
 
 	# save model architecture plot
	plot_model(model, to_file='/content/drive/MyDrive/RESEARCH-PROJECT/Multi_class_experiments/Models/model_plotcgan.png', show_shapes=True, show_layer_names=True)
	
	return model

# Load and select real data samples

Here we are selecting and returning a random sample of the real data and we assign it an initial class label for the discriminator i.e. we specify that this is a real observation, thus, belongs to y=1. 

Note: this "y=1" is to indicate a real observation, and so, here we are not indicating that it is either benign or bot. The benign or bot class is referred to as "labels".

In [ ]:
# load the real data samples
def load_real_samples(X,y):
	print(X.shape, y.shape)
	return [X, y]

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into observations and labels
    features, labels = dataset
    # choose random instances
    rand = randint(0, 1000)
    # select observations and labels
    X = features.sample(n=n_samples,random_state=rand)
    labels = labels.sample(n=n_samples,random_state=rand)
	  # generate labels
    y = ones((n_samples, 1))
    return [X, labels], y

# Generate points in the latent space

"generate_latent_points()" function takes in as an argument:


1. the size of the latent space 
2. the number of points required 
3. the number of classes i.e. 0 benign and 1 bot

Then, it returns a batch of input samples for the generator model


In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=5):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# Use points as inputs to generator 

Here, we utilise the points in the latent space as input to the generator.

In order to generate new observations we use the "generate_fake_samples()" function below. This takes as an argument:



1.   the generator model 
2.   the size of the latent space 


Then, it generates points in the latent space and uses them as input to the generator model.

Subsequently, the function returns the generated observations and their corresponding class label for the discriminator model, specifically y=0, indicating that they are fake/ generated.

In [ ]:
# use the generator to generate n fake examples, with labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

# Define training process

At this stage we are ready to fit the cGAN model.

The model is fit for 1000 training epochs and batch size of 1000 samples. These sizes are selected arbitrarily.

**The train() function takes in as arguments:**


1.   the defined generator, discriminator and cGAN models
2.   the dataset
3.   the size of the latent dimension
4.   the number of epochs and batch size 

Then, the generator model is saved after each epoch.








**The training process is performed as follows:**

1. the discriminator is updated using a half batch of real samples and then a half batch of fake samples. Thus, together they form one batch of weight updates

2.   the generator is then updated via the composite gan model

3. note: the labels are set to 1 for real and 0 for fake samples. This updates the generator towards getting better at synthesising real looking samples on the next batch

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, cgan_model, dataset, latent_dim, n_epochs=1000, n_batch=1000):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
			# generate 'fake' examples
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
			# prepare points in latent space as input for the generator
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = cgan_model.train_on_batch([z_input, labels_input], y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
	 
			if i >= 1:
			 g_model.save('/content/drive/MyDrive/RESEARCH-PROJECT/Multi_class_experiments/Generated models/multi_g_model_%04d.h5' % (i))

# Begin training process

In [ ]:
import tensorflow as tf
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim=latent_dim)
# create the cgan
cgan_model = define_cgan(g_model, d_model)
# load real observations data
dataset = load_real_samples(X_train,y_train)
# train model
train(g_model, d_model, cgan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
>933, 17/37, d1=0.097, d2=0.029 g=4.203
>933, 18/37, d1=0.137, d2=0.083 g=4.412
>933, 19/37, d1=0.131, d2=0.180 g=4.424
>933, 20/37, d1=0.172, d2=0.147 g=3.952
>933, 21/37, d1=0.167, d2=0.058 g=3.988
>933, 22/37, d1=0.168, d2=0.124 g=3.737
>933, 23/37, d1=0.123, d2=0.114 g=3.682
>933, 24/37, d1=0.168, d2=0.082 g=4.482
>933, 25/37, d1=0.110, d2=0.052 g=4.272
>933, 26/37, d1=0.086, d2=0.137 g=3.936
>933, 27/37, d1=0.115, d2=0.143 g=3.812
>933, 28/37, d1=0.198, d2=0.095 g=3.737
>933, 29/37, d1=0.140, d2=0.162 g=3.787
>933, 30/37, d1=0.161, d2=0.123 g=3.886
>933, 31/37, d1=0.107, d2=0.103 g=3.992
>933, 32/37, d1=0.123, d2=0.071 g=4.208
>933, 33/37, d1=0.109, d2=0.054 g=4.366
>933, 34/37, d1=0.107, d2=0.111 g=3.684
>933, 35/37, d1=0.139, d2=0.050 g=3.979
>933, 36/37, d1=0.137, d2=0.090 g=4.607
>933, 37/37, d1=0.105, d2=0.066 g=3.935
>934, 1/37, d1=0.122, d2=0.109 g=3.654
>934, 2/37, d1=0.157, d2=0.075 g=3.855
>934, 3/37, d1=0.161, d2=0.115 

# Find the best cGAN model post training 

Here we iterate over all saved models to assess the quality of the models. More precisely, we train a Random Forest on a generated sample and predict real data. The scores are then stored in a dictionary.

In [ ]:
# define random forest classifier
multi_cgan_rf=RandomForestClassifier(max_depth=8,n_estimators=100)

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=5):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

In [ ]:
def generate_points_per_class(latentpoints, size):
    # generate latent points 
    latent_points0, labels = generate_latent_points(latentpoints, size)
    latent_points1, labels = generate_latent_points(latentpoints, size)
    latent_points2, labels = generate_latent_points(latentpoints, size)
    latent_points3, labels = generate_latent_points(latentpoints, size)
    latent_points4, labels = generate_latent_points(latentpoints, size)
    
    # create labels
    labels_0 = asarray([0  for _ in range(size)])
    labels_1= asarray([1  for _ in range(size)])
    labels_2 = asarray([2  for _ in range(size)])
    labels_3= asarray([3  for _ in range(size)])
    labels_4= asarray([4  for _ in range(size)])
    
    return latent_points0 ,labels_0 , latent_points1, labels_1, latent_points2, labels_2, latent_points3, labels_3, latent_points4, labels_4

In [ ]:
def test_initial_quality(steps,model_path,latentpoints,size):
    scores={}
    for x in range(1,steps,1):
      # load model
      model = load_model(model_path % x)

      # generate latent points and labels 
      latent_points0 ,labels_0 , latent_points1, labels_1, latent_points2, labels_2,latent_points3, labels_3, latent_points4, labels_4 = generate_points_per_class(latentpoints, size)

      # generate data
      X_0  = model.predict([latent_points0, labels_0])
      X_1  = model.predict([latent_points1, labels_1])
      X_2  = model.predict([latent_points2, labels_2])
      X_3  = model.predict([latent_points3, labels_3])
      X_4  = model.predict([latent_points4, labels_4])

      # convert generated data into dataframes
      gen_df_0 = pd.DataFrame(data = X_0,columns = X_train.columns)
      gen_df_1 = pd.DataFrame(data = X_1,columns = X_train.columns)
      gen_df_2 = pd.DataFrame(data = X_2,columns = X_train.columns)
      gen_df_3 = pd.DataFrame(data = X_3,columns = X_train.columns)
      gen_df_4 = pd.DataFrame(data = X_4,columns = X_train.columns)
      
      # add labels to frames
      gen_df_0['attack_map']=0
      gen_df_1['attack_map']=1
      gen_df_2['attack_map']=2
      gen_df_3['attack_map']=3
      gen_df_4['attack_map']=4

      # combine dataframes together 
      df_gan = pd.concat([gen_df_0,gen_df_1,gen_df_2,gen_df_3,gen_df_4], ignore_index=True, sort=False)
   
      # train random forest model on generated data
      multi_cgan_rf.fit(df_gan.drop('attack_map',axis=1), df_gan['attack_map'])
      
      # predict real 2017 data sample
      y_pred = multi_cgan_rf.predict(X_test)

      # print classification results 
      print(classification_report(y_test,y_pred))
      
      # store results in dict
      scores.update({x: f1_score(y_test, y_pred,average='micro')})
    return scores 

# Save generated data from our best model

Finally, the best performing model is then retrieved and data generated from said model is saved for more rigorous testing of quality in a different notebook.

In [ ]:
# find the model with the highest f1 score within the scores dict
def keywithmaxval(dict1):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(dict1.values())
     k=list(dict1.keys())
     return int(k[v.index(max(v))])

In [ ]:
def save_best_data(scores,model_path,latentpoints, size,save_path):
    best_model = keywithmaxval(scores)
    model = load_model(model_path % best_model)

    # generate latent points and labels 
    latent_points0 ,labels_0 , latent_points1, labels_1, latent_points2, labels_2,latent_points3, labels_3, latent_points4, labels_4 = generate_points_per_class(latentpoints, size)

    # generate data
    X_0  = model.predict([latent_points0, labels_0])
    X_1  = model.predict([latent_points1, labels_1])
    X_2  = model.predict([latent_points2, labels_2])
    X_3  = model.predict([latent_points3, labels_3])
    X_4  = model.predict([latent_points4, labels_4])

    # convert generated data into dataframes
    gen_df_0 = pd.DataFrame(data = X_0,columns = X_train.columns)
    gen_df_1 = pd.DataFrame(data = X_1,columns = X_train.columns)
    gen_df_2 = pd.DataFrame(data = X_2,columns = X_train.columns)
    gen_df_3 = pd.DataFrame(data = X_3,columns = X_train.columns)
    gen_df_4 = pd.DataFrame(data = X_4,columns = X_train.columns)

    # add labels to frames
    gen_df_0['attack_map']=0
    gen_df_1['attack_map']=1
    gen_df_2['attack_map']=2
    gen_df_3['attack_map']=3
    gen_df_4['attack_map']=4

    # combine dataframes together 
    df_gan = pd.concat([gen_df_0,gen_df_1,gen_df_2,gen_df_3,gen_df_4], ignore_index=True, sort=False)
    
    # save best generated data 
    df_gan.to_parquet(f'{save_path}', index=False)

In [ ]:
# path where models are saved 
model_path='/content/drive/MyDrive/RESEARCH-PROJECT/Multi_class_experiments/Generated models/multi_g_model_%04d.h5'
# path where best generated data will be saved
save_path='/content/drive/MyDrive/RESEARCH-PROJECT/Multi_class_experiments/Generated data/2017Multi-cGAN.parquet'
# get performance scores 
scores = test_initial_quality(1000,model_path,100, 1000)
# save best generated dataset 
save_best_data(scores,model_path,100, 10000,save_path)

# note: as you can see, the model performance improves (up to a ceartin point) as the epoch increase.
# our best iterations can be found between : x 120 - 140

1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2500
           1       0.96      0.54      0.69      2500
           2       0.23      1.00      0.37      2500
           3       0.00      0.00      0.00      2500
           4       0.00      0.00      0.00      2500

    accuracy                           0.31     12500
   macro avg       0.24      0.31      0.21     12500
weighted avg       0.24      0.31      0.21     12500



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2500
           1       0.00      0.00      0.00      2500
           2       0.00      0.00      0.00      2500
           3       0.70      0.83      0.76      2500
           4       0.18      0.68      0.28      2500

    accuracy                           0.30     12500
   macro avg       0.18      0.30      0.21     12500
weighted avg       0.18      0.30      0.21     12500



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3
              precision    recall  f1-score   support

           0       0.99      0.12      0.22      2500
           1       0.23      1.00      0.37      2500
           2       0.00      0.00      0.00      2500
           3       0.00      0.00      0.00      2500
           4       0.00      0.00      0.00      2500

    accuracy                           0.22     12500
   macro avg       0.24      0.22      0.12     12500
weighted avg       0.24      0.22      0.12     12500



...

389
              precision    recall  f1-score   support

           0       0.30      0.91      0.46      2500
           1       0.89      0.32      0.47      2500
           2       0.00      0.00      0.00      2500
           3       0.58      0.83      0.68      2500
           4       1.00      0.21      0.35      2500

    accuracy                           0.45     12500
   macro avg       0.55      0.45      0.39     12500
weighted avg       0.55      0.45      0.39     12500

390
              precision    recall  f1-score   support

           0       0.27      0.97      0.42      2500
           1       0.94      0.32      0.47      2500
           2       0.00      0.00      0.00      2500
           3       0.96      0.82      0.88      2500
           4       1.00      0.21      0.34      2500

    accuracy                           0.46     12500
   macro avg       0.63      0.46      0.42     12500
weighted avg       0.63      0.46      0.42     12500

391
          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


392
              precision    recall  f1-score   support

           0       0.27      0.98      0.42      2500
           1       0.96      0.32      0.48      2500
           2       0.00      0.00      0.00      2500
           3       0.89      0.74      0.81      2500
           4       1.00      0.21      0.34      2500

    accuracy                           0.45     12500
   macro avg       0.62      0.45      0.41     12500
weighted avg       0.62      0.45      0.41     12500

393
              precision    recall  f1-score   support

           0       0.30      0.93      0.46      2500
           1       0.94      0.32      0.47      2500
           2       0.00      0.00      0.00      2500
           3       0.60      0.83      0.70      2500
           4       1.00      0.21      0.34      2500

    accuracy                           0.46     12500
   macro avg       0.57      0.46      0.39     12500
weighted avg       0.57      0.46      0.39     12500

394
          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


396
              precision    recall  f1-score   support

           0       0.30      0.87      0.45      2500
           1       0.91      0.32      0.47      2500
           2       0.00      0.00      0.00      2500
           3       0.59      0.77      0.67      2500
           4       0.97      0.41      0.58      2500

    accuracy                           0.47     12500
   macro avg       0.55      0.47      0.43     12500
weighted avg       0.55      0.47      0.43     12500

397
              precision    recall  f1-score   support

           0       0.24      0.92      0.38      2500
           1       0.90      0.32      0.47      2500
           2       0.00      0.00      0.00      2500
           3       0.14      0.08      0.10      2500
           4       1.00      0.21      0.34      2500

    accuracy                           0.30     12500
   macro avg       0.46      0.30      0.26     12500
weighted avg       0.46      0.30      0.26     12500

398
          